In [ ]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainModelConfig:
  model_path: Path
  root_dir: Path
  training_data: Path
  test_data_1: Path
  test_data_2: Path
  trained_model_path: Path
  batch_size: int
  learning_rate: float
  epochs: int
  train_val_split: float
  sampling_rate: int
  avg_window: int
  LSTM_window_length: int
  should_avg_be_the_feature: bool

In [ ]:
import os
print(os.getcwd())
os.chdir("../")

In [ ]:
from src.SOC_Prediction.constants import *
from src.SOC_Prediction.utils.common import read_yaml,create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        
        create_directories([self.config.artifacts_root])
        
    def train_model_config(self) -> TrainModelConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.data_dir, training.train_chemistry)
        test_data_1 = os.path.join(self.config.data_ingestion.data_dir, training.test_chemistry_1)
        test_data_2 = os.path.join(self.config.data_ingestion.data_dir, training.test_chemistry_2)
        
        create_directories([Path(training.root_dir)])

        train_model_config = TrainModelConfig(
            model_path = prepare_base_model.model_path,
            root_dir = training.root_dir,
            training_data = training_data,
            test_data_1 = test_data_1,
            test_data_2 = test_data_2,
            trained_model_path = training.trained_model_path,
            batch_size = params.prepare_base_model.BATCH_SIZE,
            learning_rate = params.prepare_base_model.LR,
            epochs = params.prepare_base_model.EPOCHS,
            train_val_split = training.train_val_split
            sampling_rate = training.sampling_rate
            avg_window = training.avg_window
            LSTM_window_length = training.LSTM_window_length
            should_avg_be_the_feature = training.should_avg_be_the_feature
            
        )

        return train_model_config

In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import scipy
import keras
import time

In [ ]:
class Training:
    def __init__(self, config: TrainModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.model_path
        )

    def train_val_test_generator(self):
        data = pd.read_csv(self.config.training_data)
        x_data = data.iloc[::self.config.sampling_rate,[1,2,3]].values
        y_data = data.iloc[::self.config.sampling_rate,[4]].values
        n_samples = len(x_data)
        
        if self.config.should_avg_be_the_feature:
            voltage = x_data[:,1]
            avg_voltage = self.rollavg_convolve_edges(voltage,self.config.avg_window).reshape(-1,1)
            current = x_data[:,0]
            avg_current = self.rollavg_convolve_edges(current,self.config.avg_window).reshape(-1,1)
            x_data = np.concatenate((avg_voltage,avg_current),axis =1)
            
        ###Write code for derivative here
        
        
        
        self.X_train = x_data[:int(self.config.n_fraction*n_samples),:]
        self.X_val = x_data[int(self.config.n_fraction*n_samples):int(0.8*n_samples),:]
        self.X_test = x_data[int(0.8*n_samples):,:]
        self.y_train = y_data[:int(self.config.n_fraction*n_samples),:]
        self.y_val = y_data[int(self.config.n_fraction*n_samples):int(0.8*n_samples),:]
        self.y_test = y_data[int(0.8*n_samples):,:]

        self.y_train = (self.y_train-np.min(self.y_train))/(np.max(self.y_train)-np.min(self.y_train))
        self.y_val = (self.y_val-np.min(self.y_val))/(np.max(self.y_val)-np.min(self.y_val))
        self.y_test = (self.y_test-np.min(self.y_test))/(np.max(self.y_test)-np.min(self.y_test))
        

        self.X_train = self.wrapper(self.X_train,self.config.window_length)
        self.X_val = self.wrapper(self.X_val,self.config.window_length)
        self.X_test = self.wrapper(self.X_test,self.config.window_length)
        self.y_train = self.y_train[:-self.config.window_length]
        self.y_val = self.y_val[:-self.config.window_length]
        self.y_test = self.y_test[:-self.config.window_length]
        
        
        self.sc = StandardScaler()
        self.X_train = self.sc.fit_transform(self.X_train)
        self.X_val = self.sc.transform(self.X_val)
        self.X_test = self.sc.transform(self.X_test)

        self.X_train = self.X_train.reshape(-1,self.config.window_length+1,x_data.shape[1])
        self.X_val  = self.X_val.reshape(-1,self.config.window_length+1,x_data.shape[1])
        self.X_test  = self.X_test.reshape(-1,self.config.window_length+1,x_data.shape[1])
        


    def rollavg_convolve_edges(self,a,n):
        assert n%2==1
        return scipy.signal.convolve(a,np.ones(n,dtype='float'), 'same')/scipy.signal.convolve(np.ones(len(a)),np.ones(n), 'same')  

        
        
    def wrapper(self,x_data, window):
        data_modified = x_data
        for i in range(1,window+1):
            data_modified = np.concatenate((data_modified,np.roll(x_data,i,axis=0)),axis = 1)

        return data_modified[:-window,:]    

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    def train(self):
        self.history = self.model.fit(
            self.X_train,
            self.y_train,
            batch_size = self.config.batch_size,
            epochs = self.config.epochs,
            validation_data = (self.X_val,self.y_val)
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [ ]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_val_test_generator()
    training.train()
    
except Exception as e:
    raise e